# เริ่มใหม่

# โหลดข้อมูลสภาพอากาศรายวัน

In [ ]:
import pandas as pd

# โหลดข้อมูลจากทุกชีทใน rain.xlsx
file_path = 'rain.xlsx'
xls = pd.ExcelFile(file_path)

df_list = []
for sheet in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet)
    df = df.rename(columns={'ปี': 'year', 'เดือน': 'month', 'วันที่': 'day'}) #เปลี่ยนชื่อไทยเป็นอิ้ง
    df['วัน'] = pd.to_datetime(df[['year', 'month', 'day']]) #สร้างตอลัมใหม่ วัน ใช้pd
    df_list.append(df) #เพ่มดาต้าเฟรมเ้าไปในลิส df list

# รวมชีททั้งหมดเข้าด้วยกันและเรียงตามวัน
weather_all = pd.concat(df_list, ignore_index=True)
weather_all = weather_all.sort_values('วัน').reset_index(drop=True)
weather_all.to_excel('rain_all_years_sorted.xlsx', index=False)
print("✅ บันทึกไฟล์ rain_all_years_sorted.xlsx เรียบร้อยแล้ว")


# โหลดข้อมูลผู้ป่วย

In [ ]:
# โหลดข้อมูลผู้ป่วยจากทุกชีท
xls = pd.ExcelFile('DHFcc12.xlsx')
df_list = []

for sheet in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet)
    df['วันเริ่มป่วย'] = pd.to_datetime(df['วันเริ่มป่วย'], errors='coerce') #ใช้ pd.to_datetime() เพื่อแปลงค่าคอลัมน์ 'วันเริ่มป่วย' เป็นชนิด datetime
    df_list.append(df)

patients = pd.concat(df_list, ignore_index=True)
patients = patients.dropna(subset=['วันเริ่มป่วย'])  # ลบแถวที่ไม่มีวันเริ่มป่วย เนา
print(f"จำนวนผู้ป่วยที่มีวันเริ่มป่วย: {len(patients)}")


# คำนวณค่าอุณหภูมิและฝนย้อนหลัง 15 วัน

In [ ]:
weather = pd.read_excel('rain_all_years_sorted.xlsx')
weather['วัน'] = pd.to_datetime(weather['วัน'])

def get_rolling_avg(date, n_days=15):
    start_date = date - pd.Timedelta(days=n_days) #หาวันที่เริ่มต้นของช่วงเวลาย้อนหลัง (15 วันก่อน date)
    mask = (weather['วัน'] >= start_date) & (weather['วัน'] < date)
    subset = weather.loc[mask] #ใช้ mask ที่ได้จากด้านบนกรองข้อมูลสภาพอากาศช่วง 15 วันย้อนหลัง
    if subset.empty:
        return pd.Series([None, None]) #ถ้าไม่มีข้อมูลส่งคืน None
    return pd.Series([
        subset['อุณหภูมิ'].mean(),
        subset['ฝน'].mean()
    ])

patients[['temp_15d_avg', 'rain_15d_avg']] = patients['วันเริ่มป่วย'].apply(get_rolling_avg)
print("✅ คำนวณค่าเฉลี่ย temp และ rain ย้อนหลัง 15 วันเสร็จสิ้น")


# สร้างตัวแปรตามเวลา (เดือน ไตรมาส ฤดูกาล)

In [ ]:
# ดึงข้อมูลปี เดือน ไตรมาส จากวันเริ่มป่วย
patients['ปี'] = patients['วันเริ่มป่วย'].dt.year
patients['เดือน'] = patients['วันเริ่มป่วย'].dt.month
patients['ไตรมาส'] = patients['วันเริ่มป่วย'].dt.quarter

# ฟังก์ชันแบ่งฤดูแบบอิงตามข้อมูลจริงของไทย
def assign_season(row):
    month = row['เดือน']
    day = row['วันเริ่มป่วย'].day

    if (month == 2 and day >= 15) or month in [3, 4] or (month == 5 and day < 15):
        return 'Summer'
    elif (month == 5 and day >= 15) or month in [6, 7, 8, 9] or (month == 10 and day < 15):
        return 'Rainy'
    else:
        return 'Winter'

# ใช้ฟังก์ชันกับแต่ละแถว
patients['ฤดูกาล'] = patients.apply(assign_season, axis=1)


# สร้างตารางสรุปรายวัน

In [ ]:
daily_counts = patients.groupby('วันพบผป').size().reset_index(name='จำนวนผู้ป่วย')

daily_weather = patients.groupby('วันพบผป').agg({
    'temp_15d_avg': 'mean',
    'rain_15d_avg': 'mean'
}).reset_index()

df_analysis = pd.merge(daily_counts, daily_weather, on='วันพบผป', how='left')


In [ ]:
df_analysis.head()


# : รวมข้อมูลอื่น ๆ (Demographic และภูมิศาสตร์)

In [ ]:
daily_vars = patients.groupby('วันพบผป').agg({
    'อายุ(ปี)': 'mean',
    'อายุ(เดือน)': 'mean',
    'เพศ': lambda x: x.mode()[0] if not x.mode().empty else None,   # หาค่าที่พบบ่อยที่สุดของเพศ
    'อาชีพ': lambda x: x.mode()[0] if not x.mode().empty else None,
    'ชื่อหมู่บ้าน': lambda x: x.mode()[0] if not x.mode().empty else None,
    'ตำบล': lambda x: x.mode()[0] if not x.mode().empty else None,
    'อำเภอ': lambda x: x.mode()[0] if not x.mode().empty else None,
    'เดือน': lambda x: x.mode()[0] if not x.mode().empty else None,
    'ปี': lambda x: x.mode()[0] if not x.mode().empty else None,
    'ไตรมาส': lambda x: x.mode()[0] if not x.mode().empty else None, # ไตรมาสที่พบบ่อยที่สุด
    'ฤดูกาล': lambda x: x.mode()[0] if not x.mode().empty else None
}).reset_index()

df_analysis = pd.merge(df_analysis, daily_vars, on='วันพบผป', how='left')


In [ ]:
print(daily_vars.head())


# ตรวจสอบความสัมพันธ์ (Correlation / Chi-Square)

In [ ]:
from scipy.stats import chi2_contingency

# --- แยกตัวแปร ---
numeric_vars = ['อายุ(ปี)_y', 'อายุ(เดือน)_y', 'temp_15d_avg', 'rain_15d_avg']
categorical_vars = ['เพศ_y', 'อาชีพ_y', 'ชื่อหมู่บ้าน_y', 'ตำบล_y', 'อำเภอ_y', 'เดือน_y', 'ปี_y', 'ไตรมาส_y', 'ฤดูกาล_y']



# --- สร้างคอลัมน์กลุ่มจำนวนผู้ป่วยแบบแบ่ง 3 กลุ่ม ---
df_analysis['จำนวนผู้ป่วยกลุ่ม'] = pd.qcut(df_analysis['จำนวนผู้ป่วย'], q=3, labels=False, duplicates='drop')

# === Correlation ===
print("=== Numeric Variables Correlation with จำนวนผู้ป่วย ===")
for var in numeric_vars:
    corr = df_analysis[var].corr(df_analysis['จำนวนผู้ป่วย'])
    print(f"{var} vs จำนวนผู้ป่วย: correlation = {corr:.4f}")

# === Chi-square test ===
print("\n=== Categorical Variables Association with จำนวนผู้ป่วย (Chi-square test) ===")
for var in categorical_vars:
    table = pd.crosstab(df_analysis[var], df_analysis['จำนวนผู้ป่วยกลุ่ม'])
    if table.shape[0] < 2 or table.shape[1] < 2:
        print(f"{var} – ข้อมูลไม่พอสำหรับ Chi-square")
        continue
    chi2, p, dof, ex = chi2_contingency(table)
    print(f"{var} vs จำนวนผู้ป่วย (กลุ่ม): p-value = {p:.4f}")


In [ ]:
print(df_analysis.columns.tolist())


# Regression Modeling (Poisson และ NB)

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

df_analysis.rename(columns={
    'จำนวนผู้ป่วย': 'cases',
    'อายุ(ปี)': 'age_year',
    'อายุ(เดือน)': 'age_month',
    'เพศ': 'gender',
    'อาชีพ': 'occupation',
    'ชื่อหมู่บ้าน': 'village',
    'ตำบล': 'sub_district',
    'อำเภอ': 'district',
    'เดือน': 'month',
    'ปี': 'year',
    'ไตรมาส': 'quarter',
    'ฤดูกาล': 'season',
    'temp_15d_avg': 'temp',
    'rain_15d_avg': 'rain'
}, inplace=True)

formula = 'cases ~ temp + rain + C(gender) + C(occupation) + C(village) + C(sub_district) + C(district) + C(month) + C(year) + C(quarter) + C(season)'

poisson_model = smf.glm(formula=formula, data=df_analysis, family=sm.families.Poisson()).fit()
print(poisson_model.summary())


# ดึงตัวแปรที่มีนัยสำคัญ

In [ ]:
import numpy as np

summary_df = poisson_model.summary2().tables[1].copy()
summary_df['exp(coef)'] = np.exp(summary_df['Coef.'])
significant_vars_df = summary_df[summary_df['P>|z|'] < 0.05]
print(significant_vars_df[['Coef.', 'P>|z|', 'exp(coef)']])


# Machine Learning ด้วย XGBoost & Random Forest

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

X = df_analysis.drop(columns=['cases', 'วันพบผป'])
y = df_analysis['cases']

X = pd.get_dummies(X, drop_first=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# --- XGBoost ---
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

print("XGBoost - MSE:", mean_squared_error(y_test, y_pred_xgb))
print("XGBoost - R2:", r2_score(y_test, y_pred_xgb))

# --- Random Forest ---
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

print("\nRandom Forest - MSE:", mean_squared_error(y_test, y_pred_rf))
print("Random Forest - R2:", r2_score(y_test, y_pred_rf))


In [ ]:
pip install joblib


In [ ]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import joblib

# 1. เตรียมข้อมูล
X = df_analysis.drop(columns=['cases', 'วันพบผป'])
y = df_analysis['cases']
X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# 2. เทรนและบันทึกโมเดล
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_model.fit(X_train, y_train)
joblib.dump(xgb_model, 'xgb_model.joblib')

rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
joblib.dump(rf_model, 'rf_model.joblib')

# 3. โหลดกลับมา (ตัวอย่าง)
xgb_model = joblib.load('xgb_model.joblib')
rf_model = joblib.load('rf_model.joblib')

# 4. ทำนายและประเมินผล
y_pred_xgb = xgb_model.predict(X_test)
y_pred_rf = rf_model.predict(X_test)

print("XGBoost - MSE:", mean_squared_error(y_test, y_pred_xgb))
print("XGBoost - R2:", r2_score(y_test, y_pred_xgb))

print("\nRandom Forest - MSE:", mean_squared_error(y_test, y_pred_rf))
print("Random Forest - R2:", r2_score(y_test, y_pred_rf))


In [ ]:
print("df_base columns:", df_base.columns.tolist())
print("df_weather columns:", df_weather.columns.tolist())


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# --- โหลดข้อมูล ---
df_base = pd.read_excel('DHFcc12.xlsx')
df_weather = pd.read_excel('rain_all_years_sorted.xlsx')

# --- แปลงวันเริ่มป่วยให้อยู่ในรูปแบบ datetime ---
df_base['วันเริ่มป่วย'] = pd.to_datetime(df_base['วันเริ่มป่วย'], errors='coerce')
df_weather['วันเริ่มป่วย'] = pd.to_datetime(df_weather['วัน'])  # ใช้วันที่เดียวกัน

# --- นับจำนวนผู้ป่วยในแต่ละวัน ---
df_count = df_base.groupby('วันเริ่มป่วย').size().reset_index(name='จำนวนผู้ป่วย')

# --- รวมตารางข้อมูลสภาพอากาศกับจำนวนผู้ป่วย ---
df_full = pd.merge(df_count, df_weather, on='วันเริ่มป่วย', how='left')

# --- สร้างฟีเจอร์จากวันที่ ---
df_full['เดือน'] = df_full['วันเริ่มป่วย'].dt.month
df_full['ปี'] = df_full['วันเริ่มป่วย'].dt.year
df_full['ไตรมาส'] = df_full['วันเริ่มป่วย'].dt.quarter

def assign_season(month):
    if month in [3, 4, 5]:
        return 'Summer'
    elif month in [6, 7, 8, 9]:
        return 'Rainy'
    else:
        return 'Winter'

df_full['ฤดูกาล'] = df_full['เดือน'].apply(assign_season)

# --- กำหนดชุดฟีเจอร์ ---
feature_sets = {
    "ข้อมูลเดิม + อุณหภูมิ": ['เดือน', 'ปี', 'ไตรมาส', 'ฤดูกาล', 'อุณหภูมิ'],
    "ข้อมูลเดิม + ความชื้น": ['เดือน', 'ปี', 'ไตรมาส', 'ฤดูกาล', 'ชื้น'],
    "ข้อมูลเดิม + ฝน": ['เดือน', 'ปี', 'ไตรมาส', 'ฤดูกาล', 'ฝน'],
    "ข้อมูลเดิม + อุณหภูมิ + ความชื้น": ['เดือน', 'ปี', 'ไตรมาส', 'ฤดูกาล', 'อุณหภูมิ', 'ชื้น'],
    "ข้อมูลเดิม + อุณหภูมิ + ฝน": ['เดือน', 'ปี', 'ไตรมาส', 'ฤดูกาล', 'อุณหภูมิ', 'ฝน'],
    "ข้อมูลเดิม + ความชื้น + ฝน": ['เดือน', 'ปี', 'ไตรมาส', 'ฤดูกาล', 'ชื้น', 'ฝน'],
    "ข้อมูลเดิม + อุณหภูมิ + ความชื้น + ฝน": ['เดือน', 'ปี', 'ไตรมาส', 'ฤดูกาล', 'อุณหภูมิ', 'ชื้น', 'ฝน'],
}

results = []

for name, features in feature_sets.items():
    df = df_full[features + ['จำนวนผู้ป่วย']].copy()

    # แปลงข้อมูลประเภท object (string) เป็นตัวเลข
    for col in df.select_dtypes(include='object').columns:
        df[col] = LabelEncoder().fit_transform(df[col].astype(str))

    X = df[features]
    y = df['จำนวนผู้ป่วย']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    results.append({'ชุดข้อมูล': name, 'RMSE': rmse, 'MAE': mae, 'R2': r2})

# แสดงผลลัพธ์
df_results = pd.DataFrame(results)
print(df_results)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor

# --- โหลดข้อมูล ---
df_base = pd.read_excel('DHFcc12.xlsx')
df_weather = pd.read_excel('rain_all_years_sorted.xlsx')

# --- แปลงวันเริ่มป่วยให้อยู่ในรูปแบบ datetime ---
df_base['วันเริ่มป่วย'] = pd.to_datetime(df_base['วันเริ่มป่วย'], errors='coerce')
df_weather['วัน'] = pd.to_datetime(df_weather['วัน'])
df_weather.rename(columns={'วัน': 'วันที่'}, inplace=True)

# --- นับจำนวนผู้ป่วยในแต่ละวัน ---
df_count = df_base.groupby('วันเริ่มป่วย').size().reset_index(name='จำนวนผู้ป่วย')

# --- คำนวณค่าเฉลี่ยย้อนหลัง 15 วัน ---
weather_agg = []
for date in df_count['วันเริ่มป่วย']:
    mask = (df_weather['วันที่'] >= date - pd.Timedelta(days=15)) & (df_weather['วันที่'] < date)
    past_weather = df_weather[mask]
    weather_agg.append({
        'วันเริ่มป่วย': date,
        'อุณหภูมิ': past_weather['อุณหภูมิ'].mean(),
        'ฝน': past_weather['ฝน'].mean(),
        'ชื้น': past_weather['ชื้น'].mean()
    })

df_weather_agg = pd.DataFrame(weather_agg)

# --- รวมข้อมูล ---
df_full = pd.merge(df_count, df_weather_agg, on='วันเริ่มป่วย', how='left')

# --- สร้างฟีเจอร์จากวันที่ ---
df_full['เดือน'] = df_full['วันเริ่มป่วย'].dt.month
df_full['ปี'] = df_full['วันเริ่มป่วย'].dt.year
df_full['ไตรมาส'] = df_full['วันเริ่มป่วย'].dt.quarter

def assign_season(month):
    if month in [3, 4, 5]:
        return 'Summer'
    elif month in [6, 7, 8, 9]:
        return 'Rainy'
    else:
        return 'Winter'

df_full['ฤดูกาล'] = df_full['เดือน'].apply(assign_season)

# --- กำหนดชุดฟีเจอร์ ---
feature_sets = {
    "ข้อมูลเดิม + อุณหภูมิ": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'อุณหภูมิ'],
    "ข้อมูลเดิม + ความชื้น": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'ชื้น'],
    "ข้อมูลเดิม + ฝน": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'ฝน'],
    "ข้อมูลเดิม + อุณหภูมิ + ความชื้น": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'อุณหภูมิ', 'ชื้น'],
    "ข้อมูลเดิม + อุณหภูมิ + ฝน": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'อุณหภูมิ', 'ฝน'],
    "ข้อมูลเดิม + ความชื้น + ฝน": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'ชื้น', 'ฝน'],
    "ข้อมูลเดิม + อุณหภูมิ + ความชื้น + ฝน": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'อุณหภูมิ', 'ชื้น', 'ฝน'],
}

results = []

for name, features in feature_sets.items():
    df = df_full[features + ['จำนวนผู้ป่วย']].copy()
    for col in df.select_dtypes(include='object').columns:
        df[col] = LabelEncoder().fit_transform(df[col].astype(str))

    X = df[features]
    y = df['จำนวนผู้ป่วย']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = XGBRegressor(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    results.append({'ชุดข้อมูล': name, 'RMSE': rmse, 'MAE': mae, 'R2': r2})

# --- แสดงผลลัพธ์ ---
df_results = pd.DataFrame(results)
df_results.sort_values('R2', ascending=False, inplace=True)
print(df_results.reset_index(drop=True))


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# ตั้ง random seed เพื่อความ reproducible
np.random.seed(42)
tf.random.set_seed(42)

# --- โหลดข้อมูล ---
df_base = pd.read_excel('DHFcc12.xlsx')
df_weather = pd.read_excel('rain_all_years_sorted.xlsx')

# --- แปลงวันเริ่มป่วยให้อยู่ในรูปแบบ datetime ---
df_base['วันเริ่มป่วย'] = pd.to_datetime(df_base['วันเริ่มป่วย'], errors='coerce')
df_weather['วัน'] = pd.to_datetime(df_weather['วัน'])
df_weather.rename(columns={'วัน': 'วันที่'}, inplace=True)

# --- นับจำนวนผู้ป่วยในแต่ละวัน ---
df_count = df_base.groupby('วันเริ่มป่วย').size().reset_index(name='จำนวนผู้ป่วย')

# --- คำนวณค่าเฉลี่ยย้อนหลัง 15 วัน ---
weather_agg = []
for date in df_count['วันเริ่มป่วย']:
    mask = (df_weather['วันที่'] >= date - pd.Timedelta(days=15)) & (df_weather['วันที่'] < date)
    past_weather = df_weather[mask]
    weather_agg.append({
        'วันเริ่มป่วย': date,
        'อุณหภูมิ': past_weather['อุณหภูมิ'].mean(),
        'ฝน': past_weather['ฝน'].mean(),
        'ชื้น': past_weather['ชื้น'].mean()
    })

df_weather_agg = pd.DataFrame(weather_agg)

# --- รวมข้อมูล ---
df_full = pd.merge(df_count, df_weather_agg, on='วันเริ่มป่วย', how='left')

# --- กำจัดแถวที่มีค่า missing (NaN) ในฟีเจอร์ (เนื่องจากค่า mean ของอดีต 15 วัน อาจมี NaN) ---
df_full.dropna(subset=['อุณหภูมิ', 'ฝน', 'ชื้น'], inplace=True)

# --- สร้างฟีเจอร์จากวันที่ ---
df_full['เดือน'] = df_full['วันเริ่มป่วย'].dt.month
df_full['ปี'] = df_full['วันเริ่มป่วย'].dt.year
df_full['ไตรมาส'] = df_full['วันเริ่มป่วย'].dt.quarter

def assign_season(month):
    if month in [3, 4, 5]:
        return 'Summer'
    elif month in [6, 7, 8, 9]:
        return 'Rainy'
    else:
        return 'Winter'

df_full['ฤดูกาล'] = df_full['เดือน'].apply(assign_season)

# --- กำหนดชุดฟีเจอร์ ---
feature_sets = {
    "ข้อมูลเดิม + อุณหภูมิ": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'อุณหภูมิ'],
    "ข้อมูลเดิม + ความชื้น": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'ชื้น'],
    "ข้อมูลเดิม + ฝน": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'ฝน'],
    "ข้อมูลเดิม + อุณหภูมิ + ความชื้น": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'อุณหภูมิ', 'ชื้น'],
    "ข้อมูลเดิม + อุณหภูมิ + ฝน": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'อุณหภูมิ', 'ฝน'],
    "ข้อมูลเดิม + ความชื้น + ฝน": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'ชื้น', 'ฝน'],
    "ข้อมูลเดิม + อุณหภูมิ + ความชื้น + ฝน": ['ปี', 'ไตรมาส', 'ฤดูกาล', 'อุณหภูมิ', 'ชื้น', 'ฝน'],
}

results = []

for name, features in feature_sets.items():
    df = df_full[features + ['จำนวนผู้ป่วย']].copy()

    # แปลง categorical เป็นเลข (LabelEncoder)
    for col in df.select_dtypes(include='object').columns:
        df[col] = LabelEncoder().fit_transform(df[col].astype(str))

    X = df[features].values
    y = df['จำนวนผู้ป่วย'].values

    # แบ่งข้อมูล train/test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Normalize features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # สร้างโมเดล Deep Learning ง่าย ๆ
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1)  # output layer สำหรับ regression
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    # Early stopping เพื่อลดการ overfit
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # train model
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0,
              validation_split=0.2, callbacks=[early_stop])

    # predict
    y_pred = model.predict(X_test).flatten()

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    results.append({'ชุดข้อมูล': name, 'RMSE': rmse, 'MAE': mae, 'R2': r2})

df_results = pd.DataFrame(results)
df_results.sort_values('R2', ascending=False, inplace=True)
print(df_results.reset_index(drop=True))
